In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import momepy as mm
import networkx as nx

In [ ]:
# https://ipython-books.github.io/147-creating-a-route-planner-for-a-road-network/

In [2]:
gdf_big = gpd.read_file('./total/gdf_line_all_n_samobl.json',
                        encoding='utf-8')

In [3]:
gdf_left = gpd.read_file('./total/gdf_line_sam_tol_new2.json',
                        encoding='utf-8')

In [4]:
gdf_all = gdf_big.append(gdf_left)
gdf_all = gdf_all.drop_duplicates(subset=['line_id', 
                                          'trip_id']).reset_index(drop=True)

In [5]:
stps_big = pd.read_csv('./total/df_stop_seq_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [144]:
reestr_big = pd.read_csv('./total/df_reestr_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [6]:
graph = gpd.read_file('./with_ped/new_graph_with_ped.json',
                        encoding='utf-8')

In [7]:
nodes = gpd.read_file('./with_ped/all_nodes_with_ped.json',
                        encoding='utf-8')

In [ ]:
one_big_line = gdf_all.iloc[[0]]


In [165]:
route_info = reestr_big[reestr_big.line_id == int(one_big_line.line_id[0])].reset_index(drop=True)
route_type = route_info.type_ts[0]

In [166]:
def get_line_graph(line, graph, nodes, route_type, buffer):
    tmp_grph = graph[((graph.type_ped == 'car_n_ped') 
                     & (graph.lanes != 0))]
    if route_type == 'tramway':
        tmp_grph = tmp_grph[((tmp_grph.type_ts == 'TM,CAR,BUS,TB,MT') 
                      | (tmp_grph.type_ts == 'TM'))].reset_index(drop=True)
    else:
        tmp_grph = tmp_grph[tmp_grph.type_ts != 'TM'].reset_index(drop=True)
    # 
    buff_one_line = gpd.GeoDataFrame(geometry=[line])
    buff_one_line.crs = graph.crs
    buff_one_line.geometry = buff_one_line.geometry.to_crs('epsg:32637').buffer(buffer).to_crs('epsg:4326')
    
    
    line_graph = gpd.sjoin(tmp_grph, buff_one_line[['geometry']], how='inner', op='within')
    line_graph = line_graph.drop('index_right', axis=1).reset_index(drop=True)
    
    lst_nodes = list(set(list(line_graph.from_node.unique()) 
                         + list(line_graph.to_node.unique())))
    #
    line_nodes = nodes[nodes.nodeID.isin(lst_nodes)]
    
    
    return line_graph, line_nodes

In [168]:
buffer=7
line = one_big_line.geometry[0]
line_graph, line_nodes = get_line_graph(line, graph, nodes, route_type, buffer)

In [169]:
i=0
segm = line[i]

In [ ]:
def get_closest_pt(ope_pt, one_grph, nodes):
    closest_link_id = mm.get_network_id(ope_pt, one_grph, 'link_id')[0]
    link_in_graph = one_grph[one_grph.link_id == closest_link_id].reset_index(drop=True)
    to_node_id = link_in_graph.to_node[0]
    from_node_id = link_in_graph.from_node[0]
    to_node = nodes[nodes.nodeID == to_node_id].reset_index(drop=True)['geometry'][0]
    from_node = nodes[nodes.nodeID == from_node_id].reset_index(drop=True)['geometry'][0]

    dist1 = ope_pt.geometry[0].distance(to_node)
    dist2 = ope_pt.geometry[0].distance(from_node)

    if dist1 < dist2:
        closest_node = to_node
        closest_node_id = to_node_id
    else:
        closest_node = from_node
        closest_node_id = from_node_id
    # 
    return closest_node,closest_node_id

In [176]:
def create_lst_coords_from_df(nodes):
    lst_geo = list(nodes.geometry)
    lst_ndid = list(nodes.nodeID)

    lst_coords=[]
    i=0
    for i in range(len(nodes)):
        lst_coords.append(lst_geo[i].coords[0])
    # 
    return lst_coords

In [177]:
def get_one_segm_seq(segm, line_graph, line_nodes):

    ope_pt = gpd.GeoDataFrame(geometry=[Point(segm.coords[0])])
    two_pt = gpd.GeoDataFrame(geometry=[Point(segm.coords[-1])])

    closest_node,closest_node_id = get_closest_pt(ope_pt, line_graph, line_nodes)
    closest_node2,closest_node_id2 = get_closest_pt(two_pt, line_graph, line_nodes)

    one_G = mm.gdf_to_nx(one_grph)
    res_lst = nx.shortest_path(one_G, closest_node.coords[0], closest_node2.coords[0])

    lst_coords = create_lst_coords_from_df(line_nodes)

    segm_seq=[]

    cnt=1
    i=0
    for i in range(len(res_lst)):
        one_lst_pt=[]
        one_coord = res_lst[i]
        j=0
        if one_coord in lst_coords:
            ind = lst_coords.index(one_coord)
            nodid = lst_ndid[ind]
        else:
            print(i)
            nodid = 0
        #
        segm_seq.append([cnt, nodid, one_coord])
        cnt+=1
    # 

    return segm_seq

In [178]:
segm_seq = get_one_segm_seq(segm, line_graph, line_nodes)

Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 47.61it/s]

Generating centroids...
Generating rtree...
Generating centroids...
Generating rtree...


In [ ]:
# def get_node_seq(line, graph, nodes, route_info):

buffer=7
# line = one_big_line
line_graph, line_nodes = get_line_graph(line, graph, nodes, route_info, buffer)

node_seq = []
i=0
for i in range(len(line)):
    segm = line[i]
    segm_seq = get_one_segm_seq(segm, line_graph, line_nodes)
    node_seq = node_seq + segm_seq
# 
# return node_seq